# 1. Import Library

# 資料準備

In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation,Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU
from keras.layers.wrappers import Bidirectional
from keras.utils import np_utils
from datetime import datetime,timedelta
import numpy as np
import json
import requests
import pandas as pd

# import urllib.request
import os
import tarfile
np.random.seed(10)

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from NLP_JiebaSegmentor import JiebaSegmentor

In [3]:
# os.chdir("../")
# os.getcwd()

In [4]:
from NLP_JiebaSegmentor import JiebaSegmentor
jieba_dict_path1 = "jieba/dict_taiwan.txt"
jieba_dict_path2 = "jieba/userdict.txt"
jieba_dict_path3 = "jieba/dict.txt.big"
jieba_dict_path4 = "jieba/dict.txt.small"
jieba_dict_path5 = "jieba/dict_system.txt"
jieba_stopwords_path = "jieba/stopwords.txt"

js = JiebaSegmentor(dict_path=jieba_dict_path1,
                    userdict=[jieba_dict_path2,
                              jieba_dict_path3,
                              jieba_dict_path4,
                              jieba_dict_path5],
                    stopwords=True,
                    stopwords_path=jieba_stopwords_path)

Building prefix dict from /project/at081-group3/Project_19999/19999_RNN/jieba/dict_taiwan.txt ...
Loading model from cache /tmp/jieba.ub496413b6051565ea73611dbd65d9084.cache
Loading model cost 0.520 seconds.
Prefix dict has been built succesfully.


In [5]:
test_cut_raw_0 = js.lcut('北京在哪呢', return_type='pandas')
test_cut_raw_0

,word
0,北京


In [6]:
#先讀取所有文章建立字典，限制字典的數量為nb_words=2000

In [7]:
df_train = pd.read_csv('data/19999_question_category_a_v14.csv')
df_test = pd.read_csv('data/19999_test_question.csv')
print(f"df_train shape : {df_train.shape}")
print(f"df_test shape : {df_test.shape}")

FileNotFoundError: File b'data/19999_question_category_a_v14.csv' does not exist

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train.info()

In [ ]:
# df = df.drop( [ 'category_b', 'category_c','category_d' ], axis = 1 )
# df.info()

In [ ]:
# # categories to index mapping
# df['category_a'] = df.category_a.astype('category').cat.codes
# df.head(10)

In [ ]:
# index to categories mapping
mapping = df_train.category.astype('category').cat.categories
mapping

In [ ]:
df_train.groupby(["category"]).size()

In [ ]:
# df_train.head(10)

In [ ]:
def cut_to_word(s):
    w_pandas = js.lcut(s, return_type='pandas')
    w_list = list(w_pandas['word'])
    combie = ''
    for i,w in enumerate(w_list):
        if w is ' ':
            continue
        combie = combie + w
        if i < len(w_list) - 1:
            combie = combie + ','
            
    return combie    

# print (cut_to_word('可以幫我聯絡'.strip()))
df_train['cut_words'] = df_train['description_clean'].apply(lambda s: cut_to_word(s.strip()))
df_test['cut_words'] = df_test['description_clean'].apply(lambda s: cut_to_word(s.strip()))

# print (cut_to_word('可以幫我聯絡'.strip()))
# df_train['cut_words'] = df_train['sentence'].apply(lambda s: cut_to_word(s.strip()))
# df_test['cut_words'] = df_test['sentence'].apply(lambda s: cut_to_word(s.strip()))

In [ ]:
# 資料打散
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)
df_train.head()

In [ ]:
# 切分資料
X = df_train['cut_words']
y = df_train['category_target']

X_test = df_test['cut_words']
y_test = df_test['category_target']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 40)

In [ ]:
# label 做 onehot
y_train_one_hot = np_utils.to_categorical(y_train)
y_val_one_hot = np_utils.to_categorical(y_val)
print (y_train_one_hot[0])
print (y_val_one_hot[0])
# 總共類別數
num_classes = len((y_train_one_hot[0]))
print ('num_classes = {}'.format(num_classes))

In [ ]:
# # 標點符號過濾
# WORD_FILTERS = '!"#$&()*+,-./:;<=>?@[\\]^_{|}~\t\n'
# 字典數量
NUM_WORDS = 5001
# 向量長度
MAX_LEN = 30

token_intent = Tokenizer(num_words=NUM_WORDS,
                        split=",")
# token_intent = Tokenizer(filters=WORD_FILTERS,
#                         num_words=NUM_WORDS,
#                         split=",")

# 斷詞後全部合併丟到fit_on_texts,組出字典
token_intent.fit_on_texts(X_train)
token_intent.fit_on_texts(X_val)

def preprocessing(x, max_len):

    #將文字轉為數字序列
    train_seq_intent = token_intent.texts_to_sequences(x)

    # 截長補短，讓所有數字序列長度一樣
    data = sequence.pad_sequences(train_seq_intent, maxlen=MAX_LEN)
    # print train_data_intent.shape
    return data

x_train = preprocessing(X_train, max_len=MAX_LEN)
x_val = preprocessing(X_val, max_len=MAX_LEN)
x_test = preprocessing(X_test, max_len=MAX_LEN)

In [ ]:
print (x_train.shape)
x_train

# 建立模型

In [ ]:
def build_model(num_classes, vocab_size, sentence_max_len):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, 
                        output_dim=32, 
                        input_length=sentence_max_len))
#     model.add(GRU(128))
    model.add(Bidirectional(LSTM(32)))
#     model.add(Bidirectional(LSTM(128,implementation=2)))
    model.add(Dropout(0.75))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    return model

model = build_model(num_classes, NUM_WORDS, MAX_LEN)
model.compile(optimizer='Adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# 訓練模型

In [ ]:
model_path = '{}.h5'.format('19999_rnn')

epochs = 50
batch_size = 100
train_ratio = 0.9

checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)
earlystop = EarlyStopping(monitor='val_loss', patience=15, verbose=1)

model_history = model.fit(x=x_train, y=y_train_one_hot,
                            batch_size=batch_size, 
                            epochs=epochs,
                            validation_split= 1 - train_ratio,
                            callbacks = [checkpoint, earlystop])

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
def show_train_history(train_history,train,validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
show_train_history(model_history,'acc','val_acc')

In [ ]:
show_train_history(model_history,'loss','val_loss')

In [ ]:
# model.save(model_path)

In [ ]:
def to_cat_name(x): 
        return mapping[x]
    
def predict(test):
    model = load_model(model_path)
    y_predict_probability = model.predict(test)
    y_predict = model.predict_classes(test)

    return to_cat_name(y_predict), y_predict, y_predict_probability

y_predict_name, y_predict, y_predict_probability = predict(x_val)

In [ ]:
answer_name = to_cat_name(y_val)

In [ ]:
model.predict_classes(x_val)

In [ ]:
mapping

In [ ]:
model.predict(x_val)

In [ ]:
predict_df = pd.DataFrame({
              '1_sentence':X_val,
              '2_y_predict':y_predict,
              '3_y_predict_name':y_predict_name,
              '4_answer':y_val,
              '5_answer_name':answer_name,
              '6_y_predict_probability':list(y_predict_probability)})
predict_df

In [ ]:
# accuracy
accuracy = 1 - (float(len(predict_df[predict_df['2_y_predict'] != predict_df['4_answer']])))/(len(predict_df))
print ('accuracy : {} !!!!!!!!'.format(accuracy))

In [ ]:
predict_df[predict_df['3_y_predict_name'] != predict_df['4_answer']]

In [ ]:
# confusion matrix
pd.crosstab(np.array(y_predict_name), np.array(answer_name),
            rownames=['predict'], colnames=['answer'])

In [ ]:
"""
自訂測試句子
"""

y_predict_name, y_predict, y_predict_probability = predict(x_test)
answer_name = to_cat_name(y_test)
predict_df = pd.DataFrame({
              '1_sentence':X_test,
              '2_y_predict':y_predict,
              '3_y_predict_name':y_predict_name,
              '4_answer':y_test,
              '5_answer_name':answer_name,
              '6_y_predict_probability':list(y_predict_probability)})

In [ ]:
# accuracy
accuracy = 1 - (float(len(predict_df[predict_df['2_y_predict'] != predict_df['4_answer']])))/(len(predict_df))
print ('accuracy : {} !!!!!!!!'.format(accuracy))

In [ ]:
# confusion matrix
pd.crosstab(np.array(y_predict_name), np.array(answer_name),
            rownames=['predict'], colnames=['answer'])

In [ ]:
"""
預測錯誤的資料觀察
"""
predict_df[predict_df['2_y_predict'] != predict_df['4_answer']]